In [1]:
!pip install --upgrade --quiet langchain langchain_community pdfplumber chromadb tqdm streamlit ollama pyngrok giskard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.9 MB/s 

In [2]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13320    0 13320    0     0  58852      0 --:--:-- --:--:-- --:--:-- 58938
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!ollama serve > rocama.log 2>&1 &
!ollama pull llama3.1 & ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest pulling manifest 
pulling 970aa74c0a90...   3% ▕▏ 8.1 MB/274 MB                  
pulling 8eeb52dfb3bb...   0% ▕▏ 3.4 MB/4.7 GB

In [4]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
llama3.1:latest            42182419e950    4.7 GB    Less than a second ago    
nomic-embed-text:latest    0a109f422b47    274 MB    56 seconds ago            


In [7]:
import os
import datetime
from tqdm import tqdm
import tempfile
import streamlit as st
from IPython.display import Markdown

import ollama
from langchain.chains import RetrievalQA
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd

In [8]:
csv_directory = "/content"
save_dir = csv_directory
vector_database_path = "/content/sample_data/vector_database"

for path in [csv_directory, vector_database_path]:
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

csv_content = []
csv_metadata = []
for csv_file in csv_files:
    file_path = os.path.join(csv_directory, csv_file)
    df = pd.read_csv(file_path)
    for _, row in df.iterrows():
        row_text = " ".join([str(item) for item in row.values])
        csv_content.append(row_text)
        metadata = {col: str(row[col]) for col in df.columns}
        csv_metadata.append(metadata)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=len, is_separator_regex=False)
text_chunks = []
chunk_metadata = []
for content, metadata in zip(csv_content, csv_metadata):
    chunks = text_splitter.split_text(content)
    text_chunks.extend(chunks)
    chunk_metadata.extend([metadata] * len(chunks))

def add_metadata(chunks, metadata_list):
    metadata_chunks = []
    for chunk, metadata in zip(chunks, metadata_list):
        metadata["author"] = "CSV Processor"
        metadata["date"] = str(datetime.date.today())
        metadata_chunks.append({"text": chunk, "metadata": metadata})
    return metadata_chunks

chunks_with_metadata = add_metadata(text_chunks, chunk_metadata)

def generate_embeddings(text_chunks, model_name='nomic-embed-text'):
    embeddings = []
    for chunk in tqdm(text_chunks, desc="Embedding text chunks"):
        embedding = ollama.embeddings(model=model_name, prompt=chunk)
        embeddings.append(embedding)
    return embeddings

texts = [chunk["text"] for chunk in chunks_with_metadata]
embeddings = generate_embeddings(texts)

documents = [Document(page_content=chunk['text'], metadata=chunk['metadata']) for chunk in chunks_with_metadata]

vector_store = Chroma.from_documents(documents=documents, embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=False), persist_directory=vector_database_path, collection_name="local-rag")
vector_store.persist()

local_model = "llama3.1"
llm = ChatOllama(model=local_model)

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newlines. Original question: {question}"""
)

retriever = MultiQueryRetriever.from_llm(vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 10}), ChatOllama(model=local_model), prompt=QUERY_PROMPT)

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

questions = 'Generate a report on all the issues in the log file'

rows = []
for document_score in vector_store.similarity_search_with_score(questions):
    rows.append((document_score[0].page_content, document_score[1]))

df = pd.DataFrame(rows, columns=['Text', 'Score'])
df

import langchain
langchain.debug = True

display(Markdown(chain.invoke({"question": questions})))

Embedding text chunks: 100%|██████████| 183/183 [00:20<00:00,  9.01it/s]
<ipython-input-8-7c3f7a83cbac>:52: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vector_store = Chroma.from_documents(documents=documents, embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=False), persist_directory=vector_database_path, collection_name="local-rag")
<ipython-input-8-7c3f7a83cbac>:53: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()
<ipython-input-8-7c3f7a83cbac>:56: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will 

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Generate a report on all the issues in the log file"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "Generate a report on all the issues in the log file"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": {
    "question": "Generate a report on all the issues in the log file"
  }
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "question": "Generate a report on all the issues in the log file"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "question": "Generate a report on all the issues i

Based on the provided content, it appears that there are no specific "issues" in the log file. However, I can help you generate a report on the events and activities recorded in the log file.

Here is a possible report based on the provided content:

**Log File Report**

**Date Range:** 81110 (no actual date specified)

**Total Events:** 4

**Events by Category:**

* **BLOCK* NameSystem.addStoredBlock**: 1 event
	+ Event ID: E6
	+ Description: blockMap updated: 10.250.7.230:50010 is added to blk_4984720043834039361 size 67108864
* **BLOCK* NameSystem.allocateBlock**: 3 events
	+ Event ID: E7 (x3)
	+ Description:
		- /user/root/randtxt/_temporary/_task_200811092030_0003_m_001506_0/part-01506. blk_1104376461837247304
		- /<*>/part-<*>. blk_<*>
		- /user/root/randtxt/_temporary/_task_200811092030_0003_m_001817_0/part-01817. blk_3291173852929756766

**Process ID and Line ID Distribution:**

* **Pid:** 28 (x2), 32, 31
* **LineId:** 250, 267, 277, 289